<a href="https://colab.research.google.com/github/redwineyards/MNIST-digit-classifier/blob/main/cnn_digitclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>